## Scrape Colour codes
Hex codes, RGB and HSL

In [1]:
from bs4 import BeautifulSoup as bs
import urllib.request as r
import re
from time import sleep
import pandas as pd

In [3]:
url_list = ['https://en.wikipedia.org/wiki/List_of_colors:_A%E2%80%93F',
           'https://en.wikipedia.org/wiki/List_of_colors:_G%E2%80%93M',
           'https://en.wikipedia.org/wiki/List_of_colors:_N%E2%80%93Z']

In [10]:
colour_list = []
for url in url_list:
    opener = r.build_opener()
    opener.addheaders = [('User-agent', 'Mozilla/5.0')]
    html = opener.open(url)
    sleep(3)
    bsObj = bs(html.read())
    bs_table = bsObj.findAll("table")
    # get corresponding table
    tbl_colour = None
    for table in bs_table:
        #if (len(table.findAll('tr')) > 2) and (len(table.findAll('tbody')) == 1):
        if (len(table.findAll('tr')) > 6):
            tbl_colour = table
            break
    #print(url + len(tbl_colour))
    if tbl_colour:
        for row in tbl_colour.findAll("tr"):
            color_name = row.find("th").getText().strip()
            row_list= []
            if len(row.findAll("td")):
                for data in row.findAll("td"):
                    if data:
                        string = data.getText().strip()
                        if len(string):
                            row_list.append(string)
                colour_list.append([color_name] + row_list)

In [11]:
colour_list

[['Absolute Zero',
  '#0048BA',
  '0%',
  '28%',
  '73%',
  '217°',
  '100%',
  '37%',
  '100%',
  '73%'],
 ['Acid green',
  '#B0BF1A',
  '69%',
  '75%',
  '10%',
  '65°',
  '76%',
  '43%',
  '76%',
  '43%'],
 ['Aero', '#7CB9E8', '49%', '73%', '91%', '206°', '70%', '70%', '47%', '91%'],
 ['Aero blue',
  '#C9FFE5',
  '79%',
  '100%',
  '90%',
  '151°',
  '100%',
  '89%',
  '100%',
  '89%'],
 ['African violet',
  '#B284BE',
  '70%',
  '52%',
  '75%',
  '288°',
  '31%',
  '63%',
  '31%',
  '75%'],
 ['Air superiority blue',
  '#72A0C1',
  '45%',
  '63%',
  '76%',
  '205°',
  '39%',
  '60%',
  '41%',
  '76%'],
 ['Alabaster',
  '#EDEAE0',
  '93%',
  '92%',
  '88%',
  '50°',
  '27%',
  '90%',
  '6%',
  '93%'],
 ['Alice blue',
  '#F0F8FF',
  '94%',
  '97%',
  '100%',
  '208°',
  '100%',
  '97%',
  '6%',
  '100%'],
 ['Alloy orange',
  '#C46210',
  '77%',
  '38%',
  '6%',
  '27°',
  '85%',
  '42%',
  '92%',
  '77%'],
 ['Almond', '#EFDECD', '94%', '87%', '80%', '30°', '52%', '87%', '14%', '94%'],

# Scrape from ColorHexa and Encycolorpedia

In [10]:
dfColor = pd.read_csv("Colours.csv")
dfColor.head()

,Name0,Desc1,Name1,HEX,R,G,B,Hue,Sat,Lum,Desc2,Name2
0,black,Black,Black,#000000,0,0,0,0,0.0,0.0,Black,Black
1,blue,medium dark blue-magenta,Navy,#000080,0,0,128,240,100.0,25.1,Dark blue,Navy blue
2,blue,medium dark blue-magenta,Darkblue,#00008B,0,0,139,240,100.0,27.3,Dark blue,Dark blue
3,blue,medium dark blue-magenta,Duke blue,#00009C,0,0,156,240,100.0,30.6,Dark blue,Duke blue
4,blue,Blue-magenta,Mediumblue,#0000CD,0,0,205,240,100.0,40.2,Strong blue,Medium blue


In [11]:
dfColor['index'] = dfColor['HEX'].apply(lambda x: x.lstrip('#'))
dfColor.set_index('index', inplace=True)
dfColor.head()

,Name0,Desc1,Name1,HEX,R,G,B,Hue,Sat,Lum,Desc2,Name2
index,,,,,,,,,,,,
000000,black,Black,Black,#000000,0,0,0,0,0.0,0.0,Black,Black
000080,blue,medium dark blue-magenta,Navy,#000080,0,0,128,240,100.0,25.1,Dark blue,Navy blue
00008B,blue,medium dark blue-magenta,Darkblue,#00008B,0,0,139,240,100.0,27.3,Dark blue,Dark blue
00009C,blue,medium dark blue-magenta,Duke blue,#00009C,0,0,156,240,100.0,30.6,Dark blue,Duke blue
0000CD,blue,Blue-magenta,Mediumblue,#0000CD,0,0,205,240,100.0,40.2,Strong blue,Medium blue


In [8]:
base_url = 'https://www.colorhexa.com/'
hexcodes = dfColor['HEX'].apply(lambda x: x.lstrip('#'))

In [9]:
hexcodes.head()

0    000000
1    000080
2    00008B
3    00009C
4    0000CD
Name: HEX, dtype: object

In [105]:
for code in hexcodes2:
    url = base_url + code
    opener = r.build_opener()
    opener.addheaders = [('User-agent', 'Mozilla/5.0')]
    try:
        html = opener.open(url)
    except:
        print(code)
        raise
    sleep(1)
    bsObj = bs(html.read())
    
    try:
        desc_text = bsObj.find("p", attrs={"class":"description"}).getText().strip()
        # regex text
        name2 = re.search("\(also known as (.+)\)", desc_text)
        if name2:
            dfColor.loc[code,'Name2'] = name2.groups()[0]
        sat = float(re.search("saturation of ([\d\.]+)%", desc_text).groups()[0])
        lum = float(re.search("lightness of ([\d\.]+)%", desc_text).groups()[0])

        desc_label = bsObj.find("div", attrs={"class":"color-description"}).getText().strip()
        desc2 = re.search(" : (.+)\.", desc_label).groups()[0]
    except:
        print(code)
    
    if desc2:
        dfColor.loc[code,'Sat'] = sat
        dfColor.loc[code,'Lum'] = lum
        dfColor.loc[code,'Desc2'] = desc2

In [72]:
hexcodes2

In [35]:
dfColor.to_csv("Colours.csv")

# Scrape from Encycolorpedia

In [14]:
base_url = 'http://encycolorpedia.com/'

In [ ]:
for code in hexcodes:
    url = base_url + code
    opener = r.build_opener()
    opener.addheaders = [('User-agent', 'Mozilla/5.0')]
    try:
        html = opener.open(url)
    except:
        print(code)
        raise
    bsObj = bs(html.read())
    info = bsObj.find("section", attrs={"id":"information"})
    
    try:
        name_texts = [x.strip() for x in info.h1.getText().split('/')]
        if name_texts[0][0] != '#':
            dfColor.loc[code,'Name1'] = name_texts[0]
        # regex text
        desc_grps = re.search(" is a (.+) shade of (.+)\. In the RGB", info.p.getText().strip())
        desc_grps = desc_grps.groups()
        dfColor.loc[code,'Desc1'] = desc_grps[0] + ' ' + desc_grps[1]
    except:
        print(code)

In [33]:
hexcodes2 = ['003366','006400','006442','0066CC','0066CD','0066FF','006994','006c7f','0070FF','0073CF','00755E','0078d7','007BB8','007FFF','00827f','0087BD','0088dc','0095B6','0096D6','00a2ed','00aaee','00B6D9','00B7EB','00B9FB','00BFFF','00c3e3','00c5cd','00CC99','00CCCC','00CCFF','00CED1','00DDDD','00dea4','00DED1','00E39A','00FA9A','00FF00','00FF3F','00FF40','00FF7F','00FFCD','00FFEF','00FFFF','010203','0247FE','02A4D3','080808','0892D0','08E8DE','0BDA51','0cbfe9','0F9DEB','0FC0FC','126180','1560BD','15f2fd','15F4EE','188bc2','18A7B5','1974D2','1AB385','1ac1dd','1c1cf0','1C39BB','1CA9C9','1CC3B4','1DACD6','1E90FF','1F75FE','1f8eed','20B2AA','214fc6','216bd6','219EBD','21ABCD','2243B6','246BCE','2496cd','24a0ed','26a7de','273be2','27A640','2887C8','29AB87','2A2F23','2A52BE','2C75FF','2D383A','2D5DA1','2D68C4','2f7532','30BA8F','30BFBF','30D5C8','3176AF','318CE7','324AB2','32ad61','32CD32','333333','343434','34b233','34b334','3560A6','35654d','357AB7','360CCC','38a32a','39569c','39A78E','39D1C2','39FF14','3AA1C1','3aa8c1','3AB09E','3AE500','3b331c','3c2126','3C69E7','3CB371','3CD070','3E8EDE','3EB489','3ECAE8','3F00FF','3FFF00','404040','4081A4','40C575','40E0D0','413628','4169E1','417DC1','42ADDE','436B95','43B3AE','446CCF','44719B','44798e','44944A','44d7a8','458AC6','45B1E8','4682B4','4682bf','46cb18','47ABCC','48929b','48BF91','48D1CC','4997D0','4aff00','4bc7cf','4C9141','4cb7a5','4CBB17','4D8C57','4d8fac','4E5180','4e82b4','4EA4BA','4F42B5','4F69C6','5072A7','50C878','5218FA','536872','536895','545AA7','56887D','56A0D3','57D200','5946B2','5A4FCF','5A7CC2','5D76CB','5D89BA','5D8AA8','5d8cae','5DA493','5F00FF','5F8A8B','5F9EA0','5FA777','5FA778','5fc9bf','6050DC','6082B6','61B834','63775b','639A8F','652DC1','6576B4','65ff00','660000','6600FF','663300','666699','669999','66B032','66b447','66c992','66FF00','66FF66','6700FF','676767','679267','68a0b0','69359c','696969','6A5ACD','6a7f7a','6B3FA0','6b9362','6BBE23','6d9a79','6D9BC3','6e7f80','6EAEA1','6F00FF','6F2DA8','6f9940','7058A3','708090','714693','714AB2','715F6D','716675','71706E','718080','71bc78','727472','734F96','7366BD','738276','738678','746CC0','749f8d','74C365','755d5b','7573A6','757575','757d75','766980','76BD17','76D7EA','7700FF','777696','778899','778BA5','7851A9','78779b','78866B','791CF8','796878','7B4BAB','7BB661','7c98ab','7CB0A1','7CFC00','7D26CD','7dc242','7E5E60','7E73B8','7eb6ff','7F00FF','7f3e98','7f6b5d','7FFF00','808080','817b69','819c8b','826b58','828e84','8359A3','836953','837050','838996','843F5B','848482','856088','856D4D','85754E','857c55','85bb65','865E55','86608E','8683A1','86aba5','875f9a','878681','87A96B','87FF2A','8806CE','88654e','88786A','896F73','89729e','898a74','8A2BE2','8A496B','8A795D','8A7F80','8A7F8D','8A9A5B','8B00FF','8B4513','8b5f4d','8B72BE','8B8589','8B8680','8bbe1b','8C7853','8C8C8C','8c9c76','8c9e5e','8D4E85','8d608c','8DA399','8db255','8E4585','8E616A','8ee53f','8F00FF','8F4155','8F509D','8F5E99','8F9779','8FD400','905D5D','906652','914E75','915C83','915F6D','918151','924E7D','926EAE','926F5B','933d41','934D91','9400D3','94417B','944747','956666','957b38','963D7F','965a3e','9678B6','967bb6','96a53c','96ff00','97645a','976638','976e9a','976F7B','9771B5','97867c','9842E3','98508E','985538','986960','987456','987654','98777B','98817B','989898','9932CC','9955BB','99637B','996666','9978B6','997A8D','9A366B','9A393D','9A4EAE','9AB973','9ACD32','9b533f','9b7653','9C5186','9C51B6','9C7687','9C7C38','9C7C3B','9c8a4d','9c8aa4','9E5E6F','9F00FF','9F4576','9f5233','9F7462','9F8170','A020F0','A03472','A0522D','a0785a','a09484','a13d2d','A17A74','A17E9A','A18178','A1C42C','A1C50A','A23B3C','A23B6C','a24f46','A3807B','a40000','a4345d','A45A52','A47D90','A4C639','A50B5E','A52A2A','A55353','A55790','A57164','A5CB0C','a62c2b','A63A79','A65E7E','A67B5B','a6d608','A757AB','A7F432','A7FC00','A83731','A8516E','A85387','A87C6D','a87ca0','A95C68','a96232','a97142','A98F64','A99A86','A9B2C3','AA00FF','AA375A','AA4069','aa6c39','AA786D','aa8736','AB274F','AB3472','AB4B52','ab4c3d','AB4E52','AB5C6D','ab6134','ABA092','AC8181','ACBF60','AD4379','AD6F69','AD8884','ADFF2F','AE2029','AE4F93','AE6938','aec6cf','AF00FF','AF4035','AF593E','AF6E4D','B03060','B04C6A','b05c52','B0665C','b07939','b08d57','b0927a','B0BC4A','B0BF1A','b0c24a','B12B7F','b14a30','B163A3','B21B1C','B22222','B31B1B','b32134','B33A7F','B3446C','b35c44','B37E9A','B38B6D','B3C110','B44668','B4745E','B53389','B55489','b56257','B5651D','b5651e','b56c60','B57281','b5a642','B5B35C','b62020','B63157','B6316C','B634BB','b64925','B768A2','B76969','B76E79','b7702d','B78727','b80ce3','B86D29','B87333','B87B6B','B8B654','B94E48','b95754','BA55D3','BA7827','ba8759','BB3385','BB466F','bb6528','bb8141','bb9351','bba46d','bbd0c9','BC13FE','BC2350','bc2d29','BC3F4A','BC5235','BD33A4','bd559c','bd9b16','bda928','BDB76B','BDD741','BDDA57','BE3262','BE4F62','be7f51','BF00FF','BF4F51','BF6935','bfbd70','BFBF6C','BFD833','BFFF00','C0362C','C0448F','C08D5E','C0C0C0','C120A2','C154C1','C18D25','C19A6B','C21E56','c23b22','C24C92','C2F732','C30B4E','C3103A','C316E9','C32148','C32350','C3272B','C39953','C3A368','C3CDE6','C400FF','C41E3A','C45655','c46210','C48E69','C51C56','C51D34','C53151','C54B8C','c57f2e','C5B356','C5B358','C65EA0','c66b27','C6930A','C6E610','c71232','C71585','C72C48','C74375','C7458B','C7FF00','C80815','c83f49','C84186','C8509B','C86D51','C88A65','c91f37','c93756','C95A49','C9AE5D','C9B93B','CA1F7B','CA2C92','ca6924','ca7309','CAE00D','CB00F5','CB410B','CB4154','cb6649','CB6D51','cb7e1f','CBA135','cc0033','CC00FF','CC3333','CC3336','cc338b','CC33CC','CC397B','CC3A71','CC474B','CC4E3C','cc4e5c','CC5555','CC7722','cc9966','CCA01D','CCCC33','cccccc','CCFF00','CD00CC','cd040b','cd071e','CD2682','CD5700','CD5B45','CD5C5C','CD607E','cd7a00','CD7F32','cd8032','CD853F','ce1127','CE2029','CE4676','CEFF00','cf0a2c','CF1020','cf142b','CF3476','cf3a24','CFB53B','d00060','D02090','D0417E','D0C0E5','D0FF14','d1001c','D10047','D10056','D1E231','D2691E','d2d9db','D2E543','D3003F','D3212D','d3419d','D34E36','D3AF37','D3D3D3','D40000','D40D7D','D4AF37','D4E646','D500B6','d57835','D65282','D68A59','D70040','D70A52','d70a53','D71868','D72D5C','D72D6D','d73b3e','D74894','D9004C','D92121','D9381E','d9463e','D94972','D9603B','D99058','d9b611','DA1884','DA1D81','DA2647','DA2C43','DA3163','DA3287','da614e','da680f','DA9100','DAA520','DADADA','DBD7D2','DC143C','dc3023','dc5349','DCDCDC','DD00FF','DD4384','DE3163','DE4C8A','de5246','DE5285','de8f4e','DF00FF','DF33EA','df6124','DFFF00','E00000','E010A8','E0115F','e0218a','E03C31','e08a1e','e08d3c','E12C2C','E18731','e1ad01','e1ad21','e1bd27','E2062C','E25822','e29c45','e2b13c','E30022','E30052','E30B5C','E30B5D','E3256B','E32636','E34234','e35c38','e3b130','E3DAC9','E3FF00','e4000f','E40078','E4007C','E44165','E48400','E49B0F','E4D00A','E4D430','E51A4C','E52B50','E52B80','E53D43','e56024','e5b73b','E5DE43','e60000','E60026','E62020','E63E62','E67E30','e69b3a','e6e0d4','e6e200','e73895','e8000d','E8192F','E86100','E936A7','e9399e','e94196','E9692C','ea3c53','EAA221','eae0c8','EB003C','EB4C42','eb593d','eb6123','ec1c24','ec2131','EC3B83','EC5800','ECD540','ECEBBD','ED1C24','ED2939','ED3CCA','ED872D','ED9121','EDEAE0','EE204D','EED202','eee600','EEED09','eeff1b','EF1DE7','EF3038','ef3939','EF9B0F','EFCC00','F00BA4','F03865','F03D6F','f04e1f','f07427','f0e130','F1B82D','F1DDCF','F2003C','F28500','f28e1c','f2b400','f2d930','f2f0e6','F34234','f35336','f3c13a','F400A1','F4C430','F4CA16','F58025','F5BD1F','F5C71A','F5E2DE','f5f3ef','f62681','f6b511','f70d1a','f7238a','f77f00','F8227F','F8B878','F8D568','f8f8f8','F94D00','fa1a8e','FADADD','FAF0E6','FAFA37','FB0081','fb4f14','fb8136','fb9902','fb9912','FC0FC0','FC5110','FCC200','FCD917','FD0E35','fd4900','fd5800','fdb900','FDBE02','fdcc0d','FDE910','FDEE00','FDF6FA','FDFD02','fdfe03','FDFF00','FE2712','FE28A2','fe5a1d','FE9D04','fecb00','FEDF00','FEE600','FEFE22','FEFE33','FF0000','ff0028','FF0038','FF003F','FF0047','FF004F','FF0057','ff006c','FF007C','FF007F','FF008F','FF0090','FF009F','FF00AF','FF00CC','FF00FF','FF033E','FF0800','FF1493','ff1b2d','FF1CB1','ff1dce','FF2052','FF2400','ff2800','ff3503','FF3800','FF3F00','ff4200','FF4500','ff4e20','FF4F00','ff5500','FF5800','ff5b00','ff5f00','FF6103','ff631c','FF6600','FF6700','FF681F','FF6FFF','ff7300','FF7518','FF7700','FF7800','FF7802','FF7A00','FF7E00','FF7F00','FF8100','ff8b00','FF8C00','FF9505','ff9900','FF9933','FF9966','FF9999','ff9a00','FF9F00','FFA000','ffa400','FFA500','ffa600','ffa631','FFA6C9','FFA700','ffa812','FFAA1D','FFAE42','FFB200','ffb300','ffb61e','FFBA00','FFBF00','ffc324','FFC40C','FFC901','ffc922','FFCC00','ffce44','FFD230','FFD300','FFD700','FFD800','FFDB00','ffdd05','FFDF00','FFE302','ffe4cd','FFE5B4','FFE816','FFEB00','FFEF00','fff000','FFF600','FFF700','FFF8DC','FFFF00','FFFF31','FFFF66','ffffbf','FFFFE0','FFFFF0','FFFFFF']

In [2]:
from selenium import webdriver

In [34]:
with webdriver.Firefox(executable_path='./WebDriver/geckodriver') as driver:

    for code in hexcodes2:
        url = base_url + code        
        try:
            driver.get(url)
            bsObj = bs(driver.page_source, 'html.parser')
        except:
            print(code)
            raise
        info = bsObj.find("section", attrs={"id":"information"})
        try:
            name_texts = [x.strip() for x in info.h1.getText().split('/')]
            if name_texts[0][0] != '#':
                dfColor.loc[code,'Name1'] = name_texts[0]
            # regex text
            desc_grps = re.search(" is a(.+)shade of (.+)\. In the RGB", info.getText().strip())
            desc_grps = desc_grps.groups()
            dfColor.loc[code,'Desc1'] = desc_grps[0] + ' ' + desc_grps[1]
        except:
            print(code)
    # Exit driver when done
    driver.quit()

In [27]:
url = 'https://encycolorpedia.com/DCDCDC'
with webdriver.Firefox(executable_path='./WebDriver/geckodriver') as driver:
    driver.get(url)
    bsObj = bs(driver.page_source, 'html.parser')
    driver.quit()

In [36]:
dfColor.sort_values('Hue',inplace=True)
dfColor.to_csv("Colours.csv")